# Modelo A para clasificación de imagenes

#### Jeffrey Daniel Leiva Cascante 2021016720
#### Richard Osvaldo León Chinchilla 2019003759

## Inicialización

In [83]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resnet es de 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    transforms.RandomRotation(20),  # Rotación aleatoria
    transforms.RandomHorizontalFlip(),  # Inversión horizontal aleatoria
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Variación de brillo y contraste
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    
])

# Cargamos el dataset de entrenamiento para cada dataset
train_dataset = datasets.ImageFolder(root='./Covid19-dataset/train', transform=transform)
train_dataset_bilateral = datasets.ImageFolder(root='./Covid19-dataset-bilateral/train', transform=transform)
train_dataset_canny = datasets.ImageFolder(root='./Covid19-dataset-canny/train', transform=transform)

# Cargamos el dataset de test para cada dataset
test_dataset = datasets.ImageFolder(root='./Covid19-dataset/test', transform=transform_test)
test_dataset_bilateral = datasets.ImageFolder(root='./Covid19-dataset-bilateral/test', transform=transform_test)
test_dataset_canny = datasets.ImageFolder(root='./Covid19-dataset-canny/test', transform=transform_test)

val_percent = 0.15 # Porcentaje de imágenes que se usarán para validación

# Calculamos el tamaño de los conjuntos de entrenamiento y validación
val_size = int(val_percent * len(train_dataset))
train_size = len(train_dataset) - val_size

val_size_bilateral = int(val_percent * len(train_dataset_bilateral))
train_size_bilateral = len(train_dataset_bilateral) - val_size_bilateral

val_size_canny = int(val_percent * len(train_dataset_canny))
train_size_canny = len(train_dataset_canny) - val_size_canny

# Se divide el dataset en dos
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
train_dataset_bilateral, val_dataset_bilateral = random_split(train_dataset_bilateral, [train_size_bilateral, val_size_bilateral])
train_dataset_canny, val_dataset_canny = random_split(train_dataset_canny, [train_size_canny, val_size_canny])

# Aplicamos el transformador de testing a los conjuntos de validación
val_dataset.dataset.transform = transform_test
val_dataset_bilateral.dataset.transform = transform_test
val_dataset_canny.dataset.transform = transform_test

# Creamos los dataloaders para cada dataset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

train_loader_bilateral = DataLoader(train_dataset_bilateral, batch_size=32, shuffle=True)
val_loader_bilateral = DataLoader(val_dataset_bilateral, batch_size=32, shuffle=False)
test_loader_bilateral = DataLoader(test_dataset_bilateral, batch_size=32, shuffle=False)

train_loader_canny = DataLoader(train_dataset_canny, batch_size=32, shuffle=True)
val_loader_canny = DataLoader(val_dataset_canny, batch_size=32, shuffle=False)
test_loader_canny = DataLoader(test_dataset_canny, batch_size=32, shuffle=False)

### Weights and Biases

In [42]:
import wandb
wandb.login(
    key="52cc20b894f4ec68c5e30b411f8f55148e7e54ec"
)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\richa\_netrc


True

## Carga de Resnet-50 preentrenado

In [84]:
import torch.nn as nn
import torchvision.models as models

# Cargamos el modelo preentrenado
resnet50  = models.resnet50(pretrained=True)

# Se modifica la capa final para adaptarla a 3 clases
num_classes = 3
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, num_classes)

# Se congela el modelo para que no se actualicen los pesos de las capas preentrenadas
for param in resnet50.parameters():
    param.requires_grad = False

for param in resnet50.fc.parameters():
    param.requires_grad = True

# Se mueve el modelo a la GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet50 = resnet50.to(device)

# Solo los parametros de la capa final se entrenan
optimizer = torch.optim.Adam(resnet50.fc.parameters(), lr=0.001)

# Se define la función de pérdida
criterion = nn.CrossEntropyLoss()


c:\Users\richa\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\richa\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Entrenamiento

In [85]:
wandb.init(project='Model-A')

wandb.config={
    'batch_size': 32,
    'learning_rate': 0.001,
    'epochs': 10,
    'optimizer': 'Adam',
}
wandb.watch(resnet50, log='all', log_freq=10)

def train_model(train_loader,val_loader,n_epochs,best_val_acc=0.0):
    for epoch in range(n_epochs):
        resnet50.train()
        train_loss = 0.0 
     
        # Entrenamiento
        for input,labels in train_loader: # Se recorren los datos de entrenamiento
            inputs, labels = input.to(device), labels.to(device)
            optimizer.zero_grad() # Se reinician los gradientes
            outputs = resnet50(inputs) # Se obtiene la salida del modelo
            loss = criterion(outputs, labels) # Se calcula la pérdida
 
            loss.backward() # Se calculan los gradientes
            optimizer.step() # Se actualizan los pesos

            train_loss += loss.item() * inputs.size(0)
            _,predicted = torch.max(outputs, 1)
            corrects = torch.sum(predicted == labels.data)

        train_loss = train_loss / len(train_loader.dataset)
        train_acc = corrects.double() / len(train_loader.dataset)

        #Validación
        resnet50.eval()
        val_loss = 0.0
        val_corrects = 0
        with torch.no_grad(): # No se calculan los gradientes en la validación
            for inputs,labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = resnet50(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

                _,predicted = torch.max(outputs, 1)
                val_corrects += torch.sum(predicted == labels.data)

            val_loss = val_loss/len(val_loader.dataset)
            val_acc = val_corrects.double() / len(val_loader.dataset)

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(resnet50.state_dict(), 'best_model.pth')
                print(f"Mejor modelo guardado con accuracy: {best_val_acc}")

            wandb.log({
                "Epoch": epoch,
                "Train Loss": train_loss,
                "Train Acc": train_acc,
                "Val Loss": val_loss,
                "Val Acc": val_acc,
            })
            print('Epoch: {} Train Loss: {:.4f} Train Acc: {:.4f} Val Loss: {:.4f} Val Acc: {:.4f}'.format(epoch, train_loss, train_acc, val_loss, val_acc))




Epoch,▁▃▆█
Train Acc,███▁
Train Loss,█▂▁▃
Val Acc,▁▁▁▁
Val Loss,▁▃█▇
Epoch,3
Train Acc,0.07583
Train Loss,0.33451
Val Acc,0.39394
Val Loss,7.30032


### Dataset Crudo

In [86]:
train_model(train_loader, val_loader, 10)

Mejor modelo guardado con accuracy: 0.4393939393939394
Epoch: 0 Train Loss: 0.9693 Train Acc: 0.0521 Val Loss: 0.9562 Val Acc: 0.4394


KeyboardInterrupt: 

In [56]:
wandb.finish()

Epoch,▁▂▃▃▄▅▆▆▇█
Train Acc,▅▆▆▅▇▃█▆▅▁
Train Loss,▄▅▆▇█▁█▄▅▃
Val Acc,▁█▇▅▇▆██▆▇
Val Loss,█▄▄▆▃▃▁▂▃▂
Epoch,9
Train Acc,0.06161
Train Loss,0.32744
Val Acc,0.91892
Val Loss,0.26672


### Dataset con filtro Bilateral

In [33]:
train_model(train_loader_bilateral, val_loader_bilateral, 10)

Mejor modelo guardado con accuracy: 0.5675675675675675
Epoch: 0 Train Loss: 0.3715 Train Acc: 0.0664 Val Loss: 1.0716 Val Acc: 0.5676
Mejor modelo guardado con accuracy: 0.8108108108108109
Epoch: 1 Train Loss: 0.2934 Train Acc: 0.0806 Val Loss: 0.5261 Val Acc: 0.8108
Mejor modelo guardado con accuracy: 0.8378378378378379
Epoch: 2 Train Loss: 0.2737 Train Acc: 0.0853 Val Loss: 0.3767 Val Acc: 0.8378
Epoch: 3 Train Loss: 0.2426 Train Acc: 0.0758 Val Loss: 0.4045 Val Acc: 0.7838
Epoch: 4 Train Loss: 0.2414 Train Acc: 0.0758 Val Loss: 0.3539 Val Acc: 0.8378
Epoch: 5 Train Loss: 0.2286 Train Acc: 0.0853 Val Loss: 0.3914 Val Acc: 0.8108
Mejor modelo guardado con accuracy: 0.8918918918918919
Epoch: 6 Train Loss: 0.2456 Train Acc: 0.0806 Val Loss: 0.2612 Val Acc: 0.8919
Epoch: 7 Train Loss: 0.2594 Train Acc: 0.0853 Val Loss: 0.2987 Val Acc: 0.8649
Mejor modelo guardado con accuracy: 0.945945945945946
Epoch: 8 Train Loss: 0.2589 Train Acc: 0.0806 Val Loss: 0.2439 Val Acc: 0.9459
Epoch: 9 Train 

In [26]:
wandb.finish()

Epoch,▁▂▃▃▄▅▆▆▇█
Train Acc,▁▇▅▄▇▇█▄█▇
Train Loss,▆█▄▄▁▅▅▂▁▄
Val Acc,▁▅▇▆▄▇█▅█▇
Val Loss,█▇▄▆▆▃▁▄▁▃
Epoch,9
Train Acc,0.08531
Train Loss,0.30809
Val Acc,0.91892
Val Loss,0.28679


### Dataset con filtro Canny

In [28]:
train_model(train_loader_canny, val_loader_canny, 10)

Mejor modelo guardado con accuracy: 0.5135135135135136
Epoch: 0 Train Loss: 0.9721 Train Acc: 0.0664 Val Loss: 1.0962 Val Acc: 0.5135
Mejor modelo guardado con accuracy: 0.5945945945945946
Epoch: 1 Train Loss: 0.8117 Train Acc: 0.0569 Val Loss: 0.8158 Val Acc: 0.5946
Mejor modelo guardado con accuracy: 0.7027027027027027
Epoch: 2 Train Loss: 0.7568 Train Acc: 0.0521 Val Loss: 0.7588 Val Acc: 0.7027
Epoch: 3 Train Loss: 0.6020 Train Acc: 0.0711 Val Loss: 0.8532 Val Acc: 0.5676
Epoch: 4 Train Loss: 0.6283 Train Acc: 0.0711 Val Loss: 0.8589 Val Acc: 0.6486
Epoch: 5 Train Loss: 0.5582 Train Acc: 0.0664 Val Loss: 0.8215 Val Acc: 0.5946
Epoch: 6 Train Loss: 0.5792 Train Acc: 0.0616 Val Loss: 0.8919 Val Acc: 0.6757
Epoch: 7 Train Loss: 0.5916 Train Acc: 0.0711 Val Loss: 0.9229 Val Acc: 0.5946
Epoch: 8 Train Loss: 0.6061 Train Acc: 0.0521 Val Loss: 0.7909 Val Acc: 0.6216
Epoch: 9 Train Loss: 0.5334 Train Acc: 0.0521 Val Loss: 0.8524 Val Acc: 0.6757


In [ ]:
wandb.finish()

## Testing

In [63]:

# Se carga el modelo
resnet50.load_state_dict(torch.load('best_model.pth'))

def test_model(test_loader):
    resnet50.eval()
    test_corrects = 0
    with torch.no_grad():
        for inputs,labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = resnet50(inputs)
            _,predicted = torch.max(outputs, 1)
            test_corrects += torch.sum(predicted == labels.data)

        test_acc = test_corrects.double() / len(test_loader.dataset)
    
        print('Test Accuracy: {:.4f}'.format(test_acc))   


C:\Users\richa\AppData\Local\Temp\ipykernel_5956\2103782149.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet50.load_state_dict(torch.load('best_model.pth'))



### Crudo

In [78]:
test_model(test_loader)

Test Accuracy: 0.3939


### Bilateral

In [34]:
test_model(test_loader_bilateral)

Test Accuracy: 0.3939
